In [1]:
import pandas as pd
import numpy as np
import pickle

We import the data from the pickle file as JSON:

In [2]:
data_json = pd.read_pickle('summoner_stats.pkl')

## Dataframes generation

We will generate two raw dataframe: one of them will contain the preffered champion statdistics and the other one the stadistics for all champions.

In [3]:
def build_dataframes(json):
    pref_list = []
    total_list = []
    for index in json:
        pref_dict = json[index]['pref_champ']['stats']
        pref_dict['summID'] = index
        pref_dict['champID'] = json[index]['pref_champ']['id']
        pref_list.append(pref_dict)
        total_dict = json[index]['total']['stats']
        total_dict['summID'] = index
        total_list.append(total_dict)
    df_pc = pd.DataFrame(pref_list).set_index('summID')
    df_total = pd.DataFrame(total_list).set_index('summID')
    return df_pc, df_total

In [4]:
df_pc, df_total = build_dataframes(data_json)

In [5]:
def put_elos(json, df):
    list_league, list_division = [], []
    for index in df.index:
        values = json[index]
        if values == '':
            league, division = [np.nan]*2
        else:
            league, division = values.split('_')
        list_league.append(league)
        list_division.append(division)
    
    df['league'] = list_league
    df['division'] = list_division
            
    return df

data_json = pd.read_pickle('summoner_elos.pkl')
df_pc = put_elos( data_json, df_pc)
df_total = put_elos( data_json, df_total)

In [7]:
df_pc = df_pc.dropna(axis=0, subset = ['league'])
df_total = df_total.dropna(axis = 0, subset = ['league'])

df_pc.league.value_counts()

platinum    8157
gold        4604
diamond     4601
silver      1376
bronze       470
master        27
Name: league, dtype: int64

In [8]:
from sklearn import preprocessing
encoLeague, encoDiv = preprocessing.LabelEncoder(), preprocessing.LabelEncoder()
encoLeague.fit(['bronze', 'silver', 'gold', 'platinum', 'diamond', 'master'])
print('League Encoder Classes: ', encoLeague.classes_)
encoDiv.fit(['1', '2', '3', '4', '5'])
print('Division Encoder Classes: ', encoDiv.classes_)

df_pc['league'] = encoLeague.transform(df_pc['league'])
df_total['league'] = encoLeague.transform(df_total['league'])

df_pc['division'] = encoDiv.transform(df_pc['division'])
df_total['division'] = encoDiv.transform(df_total['division'])

League Encoder Classes:  ['bronze' 'diamond' 'gold' 'master' 'platinum' 'silver']
Division Encoder Classes:  ['1' '2' '3' '4' '5']


We inspect the columns for the 'Preferred champion" dataframe. We also set the "Summoner ID" as the index. 

In [ ]:
print(df_pc.columns)
df_pc.head()

We reorder the columns and drop some of them like the 0 value stats (deprecated data from the Riot API) and unrelevant data ('unreal kills', 'games against IA', etc..)

In [ ]:
df_pc = df_pc[['champID', 'totalSessionsPlayed', 'totalSessionsWon', 'totalSessionsLost', 'totalGoldEarned', 
               'totalMinionKills', 'totalTurretsKilled', 'totalPhysicalDamageDealt', 'totalMagicDamageDealt', 
               'totalDamageTaken', 'totalDeathsPerSession', 'maxNumDeaths', 'totalAssists', 'totalChampionKills', 'totalDoubleKills', 
               'totalTripleKills', 'totalQuadraKills', 'totalPentaKills', 'mostChampionKillsPerSession', 'league']]
print(df_pc.dtypes)
df_pc.head()

In [ ]:
print(df_pc.columns)
df_pc.head()

Now we perform the same actions for the 'All champions' dataframe.

In [ ]:
df_total = df_total[['totalSessionsPlayed', 'totalSessionsWon', 'totalSessionsLost', 'totalGoldEarned', 
               'totalMinionKills', 'totalNeutralMinionsKilled', 'totalTurretsKilled', 'totalPhysicalDamageDealt', 
                'totalMagicDamageDealt', 'totalDamageTaken', 'totalHeal', 'totalDeathsPerSession', 'totalAssists', 
                'totalChampionKills', 'totalDoubleKills', 'totalTripleKills', 'totalQuadraKills', 'totalPentaKills',
                    'league']]
print(df_total.dtypes)
df_total.head()

## Data filtering and standarization by sessions played

We only consider the accounts that have 50 or more games played. 

In [ ]:
df_pc = df_pc[df_pc.totalSessionsPlayed >= 50]
df_total = df_total[df_total.totalSessionsPlayed >= 50]

The "total" stadisitics are calculated as the sum of the totalSessionsPlayed so we must calculate the average data for all games.

In [ ]:
df_pc['GoldEarned'] = df_pc['totalGoldEarned'] / df_pc['totalSessionsPlayed']
df_pc['MinionKills'] = df_pc['totalMinionKills'] / df_pc['totalSessionsPlayed']             
df_pc['PhysicalDamageDealt'] = df_pc['totalPhysicalDamageDealt'] / df_pc['totalSessionsPlayed']     
df_pc['MagicDamageDealt'] = df_pc['totalMagicDamageDealt'] / df_pc['totalSessionsPlayed']  
df_pc['DamageTaken'] = df_pc['totalDamageTaken'] / df_pc['totalSessionsPlayed']              
df_pc['DoubleKills'] = df_pc['totalDoubleKills'] / df_pc['totalSessionsPlayed']
df_pc['TripleKills'] = df_pc['totalTripleKills'] / df_pc['totalSessionsPlayed']
df_pc['QuadraKills'] = df_pc['totalQuadraKills'] / df_pc['totalSessionsPlayed']
df_pc['PentaKills'] = df_pc['totalPentaKills'] / df_pc['totalSessionsPlayed']
df_pc['TurretsKilled'] = df_pc['totalTurretsKilled'] / df_pc['totalSessionsPlayed']

df_total['GoldEarned'] = df_total['totalGoldEarned'] / df_total['totalSessionsPlayed']
df_total['MinionKills'] = df_total['totalMinionKills'] / df_total['totalSessionsPlayed']             
df_total['PhysicalDamageDealt'] = df_total['totalPhysicalDamageDealt'] / df_total['totalSessionsPlayed']     
df_total['MagicDamageDealt'] = df_total['totalMagicDamageDealt'] / df_total['totalSessionsPlayed']
df_total['DamageTaken'] = df_total['totalDamageTaken'] / df_total['totalSessionsPlayed']
df_total['DoubleKills'] = df_total['totalDoubleKills'] / df_total['totalSessionsPlayed']
df_total['TripleKills'] = df_total['totalTripleKills'] / df_total['totalSessionsPlayed']
df_total['QuadraKills'] = df_total['totalQuadraKills'] / df_total['totalSessionsPlayed']
df_total['PentaKills'] = df_total['totalPentaKills'] / df_total['totalSessionsPlayed']
df_total['TurretsKilled'] = df_total['totalTurretsKilled'] / df_total['totalSessionsPlayed']
df_total['Heal'] = df_total['totalHeal'] / df_total['totalSessionsPlayed']
df_total['NeutralMinionsKilled	'] = df_total['totalNeutralMinionsKilled'] / df_total['totalSessionsPlayed']

### KDA calculation

One important metric in competitive games as League of Legends is the KDA (Kills-Deaths-Assists) ratio. For a given player when can calculate the KDA for the ranked Season 5 as 
\begin{equation}
KDA = \frac{totalChampionKills + totalAssists}{totalDeathsPerSession} 
\end{equation}

In [ ]:
df_pc['KDA'] = (df_pc['totalChampionKills'] + df_pc['totalAssists']) / df_pc['totalDeathsPerSession']

In [ ]:
df_pc[['KDA', 'totalChampionKills', 'totalAssists', 'totalDeathsPerSession']].head()

In [ ]:
df_total['totalKDA'] = (df_total['totalChampionKills'] + df_total['totalAssists']) / df_total['totalDeathsPerSession']

In [ ]:
df_total[['totalKDA', 'totalChampionKills', 'totalAssists', 'totalDeathsPerSession']].head()

### Win rate calculation

We calculate the Win Rate percentage per player for both dataframes (it will be the same as they are the same players).

In [ ]:
df_pc['WinRate'] = df_pc['totalSessionsWon'] / df_pc['totalSessionsPlayed']

In [ ]:
df_pc[['WinRate', 'totalSessionsPlayed', 'totalSessionsWon', 'totalSessionsLost']].head()

In [ ]:
df_total['WinRate'] = df_pc['totalSessionsWon'] / df_pc['totalSessionsPlayed']

In [ ]:
df_total[['WinRate', 'totalSessionsPlayed', 'totalSessionsWon', 'totalSessionsLost']].head()

### Drop the unnecessary data

In [ ]:
df_pc.drop(df_pc[['totalChampionKills', 'totalTurretsKilled', 'totalAssists', 'totalDeathsPerSession', 'totalSessionsPlayed', 'totalSessionsWon', 'totalSessionsLost', 'totalGoldEarned', 'totalMinionKills', 'totalPhysicalDamageDealt', 'totalMagicDamageDealt','totalDamageTaken', 'totalDoubleKills', 'totalTripleKills', 'totalQuadraKills', 'totalPentaKills']], axis=1, inplace=True)

In [ ]:
df_total.drop(df_total[['totalHeal' ,'totalNeutralMinionsKilled','totalChampionKills', 'totalTurretsKilled', 'totalAssists', 'totalDeathsPerSession', 'totalSessionsPlayed', 'totalSessionsWon', 'totalSessionsLost', 'totalGoldEarned', 'totalMinionKills', 'totalPhysicalDamageDealt', 'totalMagicDamageDealt','totalDamageTaken', 'totalDoubleKills', 'totalTripleKills', 'totalQuadraKills', 'totalPentaKills']], axis=1, inplace=True)

## Dataframe description 

In [ ]:
df_pc.head()

In [ ]:
df_total.head()

### Mean, std, min, max and percentiles

In [ ]:
df_pc.describe()

In [ ]:
df_total.describe()

### Covariances and Correlation between features

In [ ]:
df_pc.cov()

In [ ]:
df_pc.corr()

In [ ]:
df_total.cov()

In [ ]:
df_total.corr()

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

py.sign_in('juanluismoralperez', 'MUMjWF4QwNy8AtZW6WUq')

trace1 = go.Scatter3d(
    x=df_pc.PhysicalDamageDealt,
    y=df_pc.KDA,
    z=df_pc.league,
    mode='markers',
    marker=dict(
        size=12,
        color=df_pc.league,                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)

data = [trace1]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='3d-scatter-colorscale')


## Data visualization by champion 

In [ ]:
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.set(style="whitegrid", color_codes=True)

In [ ]:
fig = plt.figure(num=None, figsize=(10, 6), dpi=80, facecolor='w', edgecolor='k')
ax = fig.add_subplot(1,1,1)
ax.scatter(df_pc['champID'], df_pc['KDA'])
plt.xlabel('ChampID')
plt.ylabel('KDA')
plt.xlim(0, 440)
plt.ylim(0, 10.5)
plt.show()

In [ ]:
fig = plt.figure(num=None, figsize=(10, 6), dpi=80, facecolor='w', edgecolor='k')
ax = fig.add_subplot(1,1,1)
ax.scatter(df_pc['champID'], df_pc['WinRate'])
plt.xlabel('ChampID')
plt.ylabel('WinRate(%)')
plt.xlim(0, 440)
plt.ylim(0, 1)
plt.show()

In [ ]:
fig = plt.figure(num=None, figsize=(10, 6), dpi=80, facecolor='w', edgecolor='k')
ax = fig.add_subplot(1,1,1)
ax.scatter(df_pc['champID'], df_pc['GoldEarned'])
plt.xlabel('ChampID')
plt.ylabel('Average Gold Earned')
plt.xlim(-1, 440)
plt.ylim(6500, 18000)
plt.show()

In [ ]:
fig = plt.figure(num=None, figsize=(10, 6), dpi=80, facecolor='w', edgecolor='k')
ax = fig.add_subplot(1,1,1)
ax.scatter(df_pc['champID'], df_pc['MinionKills'])
plt.xlabel('ChampID')
plt.ylabel('Creep Score (CS)')
plt.xlim(-1, 440)
plt.ylim(-0.05, 300)
plt.show()

In [ ]:
fig = plt.figure(num=None, figsize=(18, 6), dpi=80, facecolor='w', edgecolor='k')
ax1 = fig.add_subplot(1,3,1)
ax1.scatter(df_pc['champID'], df_pc['PhysicalDamageDealt'])
ax2 = fig.add_subplot(1,3,2)
ax2.scatter(df_pc['champID'], df_pc['MagicDamageDealt'])
ax3 = fig.add_subplot(1,3,3)
ax3.scatter(df_pc['champID'], df_pc['DamageTaken'])
ax1.set_xlim([-1, 440])
ax1.set_ylim([0, 300000])
ax1.set_xlabel('ChampID')
ax1.set_ylabel('Physical Dmg')
ax2.set_xlim([-1, 440])
ax2.set_ylim([0, 250000])
ax2.set_xlabel('ChampID')
ax2.set_ylabel('Magic Dmg')
ax3.set_xlim([-1, 440])
ax3.set_ylim([0, 55000])
ax3.set_xlabel('ChampID')
ax3.set_ylabel('Dmg taken')
plt.show()

In [ ]:
fig = plt.figure(num=None, figsize=(10, 6), dpi=80, facecolor='w', edgecolor='k')
ax1 = fig.add_subplot(2,2,1)
ax1.scatter(df_pc['champID'], df_pc['PentaKills'])
ax2 = fig.add_subplot(2,2,2)
ax2.scatter(df_pc['champID'], df_pc['QuadraKills'])
ax3 = fig.add_subplot(2,2,3)
ax3.scatter(df_pc['champID'], df_pc['TripleKills'])
ax4 = fig.add_subplot(2,2,4)
ax4.scatter(df_pc['champID'], df_pc['DoubleKills'])
ax1.set_xlim([-0.5, 440])
ax1.set_ylim([0, 0.2])
ax1.set_xlabel('ChampID')
ax1.set_ylabel('Pentakills')
ax2.set_xlim([-0.5, 440])
ax2.set_ylim([0, 0.35])
ax2.set_xlabel('ChampID')
ax2.set_ylabel('Quadrakills')
ax3.set_xlim([-0.5, 440])
ax3.set_ylim([0, 1])
ax3.set_xlabel('ChampID')
ax3.set_ylabel('Triplekills')
ax4.set_xlim([-0.5, 440])
ax4.set_ylim([0, 3.5])
ax4.set_xlabel('ChampID')
ax4.set_ylabel('Doublekills')
plt.show()

In [ ]:
from pandas.tools.plotting import scatter_matrix
scatter_matrix(df_pc, figsize = (20,20))
plt.show()

## Data visualization by player (all champions played)

In [ ]:
scatter_matrix(df_total, figsize = (20,20))
plt.show()